In [ ]:
# YOLOv13 Custom Dataset Training
## Real-Time Object Detection with Hypergraph-Enhanced Adaptive Visual Perception

This notebook demonstrates how to train YOLOv13 on custom datasets using the Roboflow structure.

**Paper**: [YOLOv13: Real-Time Object Detection with Hypergraph-Enhanced Adaptive Visual Perception](https://github.com/iMoonLab/yolov13)

### Key Features:
- Hypergraph-enhanced adaptive visual perception
- Superior performance compared to YOLOv11/v12
- Flash Attention acceleration support
- Roboflow integration for seamless dataset management


In [ ]:
## 1. Environment Setup
Install required dependencies including Flash Attention for acceleration


In [ ]:
# Install Flash Attention (Linux x86_64 with CUDA 11)
!wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl
!pip install flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl


In [ ]:
# Clone YOLOv13 repository
!git clone https://github.com/iMoonLab/yolov13.git
%cd yolov13


In [ ]:
# Install YOLOv13 and dependencies
!pip install -r requirements.txt
!pip install -e .
!pip install roboflow supervision


In [ ]:
## 2. Import Required Libraries


In [ ]:
import os
import yaml
import torch
from ultralytics import YOLO
from roboflow import Roboflow
import supervision as sv
from IPython.display import Image, display
import matplotlib.pyplot as plt
import cv2
import numpy as np

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name()}")


In [ ]:
## 3. Dataset Preparation with Roboflow
### Option A: Download from Roboflow Universe


In [ ]:
# Initialize Roboflow (get your API key from https://roboflow.com/)
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")

# Download dataset (replace with your project details)
project = rf.workspace("your-workspace").project("your-project")
dataset = project.version(1).download("yolov8")  # YOLOv13 uses YOLOv8 format

dataset_path = dataset.location
print(f"Dataset downloaded to: {dataset_path}")


In [ ]:
### Option B: Use Your Local Dataset
Ensure your dataset follows this structure:
```
dataset/
├── train/
│   ├── images/
│   └── labels/
├── valid/
│   ├── images/
│   └── labels/
├── test/
│   ├── images/
│   └── labels/
└── data.yaml
```


In [ ]:
# If using local dataset, specify the path
# dataset_path = "/path/to/your/dataset"

# Verify dataset structure
def verify_dataset_structure(dataset_path):
    required_dirs = ['train/images', 'train/labels', 'valid/images', 'valid/labels']
    for dir_path in required_dirs:
        full_path = os.path.join(dataset_path, dir_path)
        if os.path.exists(full_path):
            print(f"✓ {dir_path}: {len(os.listdir(full_path))} files")
        else:
            print(f"✗ {dir_path}: Missing")
    
    data_yaml = os.path.join(dataset_path, 'data.yaml')
    if os.path.exists(data_yaml):
        print(f"✓ data.yaml: Found")
        with open(data_yaml, 'r') as f:
            data = yaml.safe_load(f)
            print(f"  Classes: {data.get('nc', 'Not specified')}")
            print(f"  Names: {data.get('names', 'Not specified')}")
    else:
        print(f"✗ data.yaml: Missing")

verify_dataset_structure(dataset_path)


In [ ]:
## 4. Model Selection and Configuration
Choose the appropriate YOLOv13 model variant based on your requirements


In [ ]:
# YOLOv13 model variants and their characteristics
model_configs = {
    'yolov13n': {
        'config': 'yolov13n.yaml',
        'pretrained': 'yolov13n.pt',
        'params': '2.3M',
        'map50-95': '37.3',
        'speed': '1.25ms',
        'description': 'Nano - Fastest, smallest model'
    },
    'yolov13s': {
        'config': 'yolov13s.yaml',
        'pretrained': 'yolov13s.pt',
        'params': '20.8M',
        'map50-95': '48.0',
        'speed': '2.98ms',
        'description': 'Small - Good balance of speed and accuracy'
    },
    'yolov13l': {
        'config': 'yolov13l.yaml',
        'pretrained': 'yolov13l.pt',
        'params': '88.4M',
        'map50-95': '53.4',
        'speed': '8.63ms',
        'description': 'Large - High accuracy, moderate speed'
    },
    'yolov13x': {
        'config': 'yolov13x.yaml',
        'pretrained': 'yolov13x.pt',
        'params': '199.2M',
        'map50-95': '54.8',
        'speed': '14.67ms',
        'description': 'Extra Large - Highest accuracy, slower speed'
    }
}

print("YOLOv13 Model Variants:")
print("-" * 80)
for model, info in model_configs.items():
    print(f"{model.upper():10} | Params: {info['params']:8} | mAP: {info['map50-95']:5} | Speed: {info['speed']:8} | {info['description']}")

# Select model (change this based on your requirements)
selected_model = 'yolov13s'  # Recommended for most use cases
model_config = model_configs[selected_model]

print(f"\nSelected model: {selected_model.upper()}")
print(f"Description: {model_config['description']}")


In [ ]:
## 5. Training Configuration
Set up hyperparameters optimized for YOLOv13


In [ ]:
# Training hyperparameters (optimized for YOLOv13)
training_config = {
    # Basic settings
    'epochs': 100,  # Increase to 300-600 for production
    'batch': 16,    # Adjust based on GPU memory (32, 64, 128, 256)
    'imgsz': 640,   # Image size
    
    # Model-specific hyperparameters (from YOLOv13 paper)
    'scale': 0.5 if selected_model == 'yolov13n' else 0.9,  # Scaling factor
    
    # Data augmentation
    'mosaic': 1.0,  # Mosaic augmentation probability
    'mixup': 0.0 if selected_model == 'yolov13n' else {
        'yolov13s': 0.05,
        'yolov13l': 0.15,
        'yolov13x': 0.2
    }.get(selected_model, 0.0),
    
    'copy_paste': 0.1 if selected_model == 'yolov13n' else {
        'yolov13s': 0.15,
        'yolov13l': 0.5,
        'yolov13x': 0.6
    }.get(selected_model, 0.1),
    
    # Optimization
    'optimizer': 'AdamW',
    'lr0': 0.01,     # Initial learning rate
    'lrf': 0.01,     # Final learning rate
    'momentum': 0.937,
    'weight_decay': 0.0005,
    
    # Hardware
    'device': '0',   # GPU device (0, 1, 2, 3 or '0,1,2,3' for multi-GPU)
    'workers': 8,    # Number of worker threads
    
    # Validation
    'val': True,
    'save_period': 10,  # Save checkpoint every N epochs
    
    # Advanced
    'amp': True,     # Automatic Mixed Precision
    'half': False,   # Use FP16 inference
}

print("Training Configuration:")
print("-" * 40)
for key, value in training_config.items():
    print(f"{key:15}: {value}")


In [ ]:
## 6. Initialize YOLOv13 Model


In [ ]:
# Initialize YOLOv13 model
try:
    # Try to load pretrained model first
    model = YOLO(model_config['pretrained'])
    print(f"✓ Loaded pretrained {selected_model.upper()} model")
except:
    # If pretrained model not available, load from config
    model = YOLO(model_config['config'])
    print(f"✓ Loaded {selected_model.upper()} model from config (no pretrained weights)")

# Print model info
print(f"\nModel Summary:")
print(f"Parameters: {sum(p.numel() for p in model.model.parameters()):,}")
print(f"Layers: {len(list(model.model.modules()))}")


In [ ]:
## 7. Start Training
Train YOLOv13 with hypergraph-enhanced adaptive visual perception


In [ ]:
# Start training
print(f"Starting YOLOv13 training on custom dataset...")
print(f"Model: {selected_model.upper()}")
print(f"Dataset: {dataset_path}")
print(f"Epochs: {training_config['epochs']}")
print(f"Batch size: {training_config['batch']}")
print("-" * 60)

# Train the model
results = model.train(
    data=data_yaml_path,
    epochs=training_config['epochs'],
    batch=training_config['batch'],
    imgsz=training_config['imgsz'],
    scale=training_config['scale'],
    mosaic=training_config['mosaic'],
    mixup=training_config['mixup'],
    copy_paste=training_config['copy_paste'],
    optimizer=training_config['optimizer'],
    lr0=training_config['lr0'],
    lrf=training_config['lrf'],
    momentum=training_config['momentum'],
    weight_decay=training_config['weight_decay'],
    device=training_config['device'],
    workers=training_config['workers'],
    val=training_config['val'],
    save_period=training_config['save_period'],
    amp=training_config['amp'],
    project='yolov13_training',
    name='custom_dataset',
    exist_ok=True
)

print("\n" + "="*60)
print("TRAINING COMPLETED!")
print("="*60)


In [ ]:
## 8. Evaluate Model Performance


In [ ]:
# Evaluate on validation set
print("Evaluating model performance...")
metrics = model.val(data=data_yaml_path)

# Print key metrics
print(f"\nValidation Results:")
print("-" * 40)
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP75: {metrics.box.map75:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")


In [ ]:
## 9. Export Model for Deployment


In [ ]:
# Load the best trained model
best_model = YOLO('yolov13_training/custom_dataset/weights/best.pt')

# Export model to different formats
print("Exporting model for deployment...")

# Export to ONNX (recommended for most deployments)
onnx_path = best_model.export(format="onnx", half=False)
print(f"✓ ONNX model exported: {onnx_path}")

# Export to TensorRT (for NVIDIA GPUs)
try:
    trt_path = best_model.export(format="engine", half=True)
    print(f"✓ TensorRT model exported: {trt_path}")
except:
    print("⚠ TensorRT export failed (requires TensorRT installation)")

# Export to CoreML (for Apple devices)
try:
    coreml_path = best_model.export(format="coreml")
    print(f"✓ CoreML model exported: {coreml_path}")
except:
    print("⚠ CoreML export failed (requires coremltools)")

print("\nModel export completed!")


In [ ]:
## 10. Test Inference and Results


In [ ]:
# Test on sample images
def test_inference(model, test_images_path, confidence=0.25):
    """Test model on custom images"""
    if os.path.exists(test_images_path):
        image_files = [f for f in os.listdir(test_images_path) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for img_file in image_files[:4]:  # Test on first 4 images
            img_path = os.path.join(test_images_path, img_file)
            
            # Run inference
            results = model.predict(img_path, conf=confidence)
            
            # Display results
            for r in results:
                im_array = r.plot()  # plot a BGR numpy array of predictions
                im_array = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB)
                
                plt.figure(figsize=(12, 8))
                plt.imshow(im_array)
                plt.title(f'YOLOv13 Predictions: {img_file}')
                plt.axis('off')
                plt.show()
                
                # Print detection results
                if len(r.boxes) > 0:
                    print(f"\nDetections in {img_file}:")
                    for box in r.boxes:
                        cls = int(box.cls)
                        conf = float(box.conf)
                        class_name = model.names[cls] if cls < len(model.names) else f"Class_{cls}"
                        print(f"  {class_name}: {conf:.2f}")
                else:
                    print(f"No detections in {img_file}")
    else:
        print(f"Test images path not found: {test_images_path}")

# Test on validation images
test_images_path = os.path.join(dataset_path, 'valid/images')
test_inference(best_model, test_images_path)


In [ ]:
## 11. Summary and Next Steps


In [ ]:
# Training summary
print("🎉 YOLOv13 Custom Training Complete!")
print("=" * 50)
print(f"Model: {selected_model.upper()}")
print(f"Dataset: {dataset_path}")
print(f"Training epochs: {training_config['epochs']}")
print(f"Best model saved: yolov13_training/custom_dataset/weights/best.pt")
print(f"Last model saved: yolov13_training/custom_dataset/weights/last.pt")

print("\n📊 Performance Metrics:")
try:
    print(f"mAP50-95: {metrics.box.map:.3f}")
    print(f"mAP50: {metrics.box.map50:.3f}")
    print(f"Precision: {metrics.box.mp:.3f}")
    print(f"Recall: {metrics.box.mr:.3f}")
except:
    print("Metrics not available")

print("\n🚀 Next Steps:")
print("1. Fine-tune hyperparameters for better performance")
print("2. Increase training epochs (300-600) for production")
print("3. Collect more data if mAP is below target")
print("4. Deploy model using exported formats (ONNX/TensorRT)")
print("5. Monitor model performance in production")

print("\n📚 Key Features of YOLOv13:")
print("• Hypergraph-enhanced adaptive visual perception")
print("• Superior performance vs YOLOv11/v12")
print("• Flash Attention acceleration support")
print("• Optimized for real-time object detection")

print("\n📖 Citation:")
print("Lei, M., Li, S., Wu, Y., et al. (2025). YOLOv13: Real-Time Object Detection")
print("with Hypergraph-Enhanced Adaptive Visual Perception. arXiv:2506.17733")
